### Project Overview
You are building a Proof of Concept (PoC) for a Knowledge Graph-based NLP Chatbot for Material Safety Data Sheets (MSDS). The PoC involves:

Extracting data from an MSDS PDF using the PyPDF2 library.
Parsing the extracted data to generate RDF tuples using NLP techniques with the spaCy library.
Utilizing the RDF schema provided in the msds_rdf.ttl file to structure the RDF tuples.

1. Set Up the Python Environment </br>
<br>python3 -m venv .venv
<br>source .venv/bin/activate   

In [3]:
# Create a requirements.txt file with the installed packages
!pip freeze > requirements.txt

In [ ]:
from pdf_processor import cl_process_pdf
from langchain.indexes.graph import GraphIndexCreator
from prompt_generator import cl_msds_Ontology_PromptGenerator


2. Extract Data from the PDF
Use the PyPDF2 library to extract text from the MSDS PDF file.

In [14]:
# Step 1: Create instance of read_pdf
path = "/Users/I310202/Library/CloudStorage/OneDrive-SAPSE/SR@Work/81.Innovations/98.AI_Developments/33.AI_MSDS/Build_MSDS_SAPKGE/Documents/WD-40.pdf"
pdf_reader = cl_process_pdf(path)

# Step 2: Extract text from the PDF
pdf_doc = pdf_reader.load_documents()
pdf_text = pdf_reader.extract_text()

# Step 3: Sanitize the extracted text
pdf_text = pdf_reader.sanitize_text(pdf_text)

# Step 4: Create Chunks
chunks = pdf_reader.create_chunks(pdf_doc, chunk_size=1000, chunk_overlap=200)

# Step 5: Process documents in parallel to extract graph documents
print(f"total chunks: {len(chunks)}")

total chunks: 5


### 3. Extract SDS Ontology

In [6]:
def read_ontology(file_path):
    with open(file_path, 'r') as file:
        return file.read()

sds_ontology = read_ontology("msds_ontology.ttl")

### 3. Initialize SAP AI Core Foundation model and Invoke the Same

In [17]:
aic_config = {
    "aic_client_id": "sb-367079b3-3023-4300-8c56-7f05b812bbe5!b122220|aicore!b164",
    "aic_client_secret": "664eed88-a51d-40d3-aa44-d99cd7ff09f2$GfLXh5QaFHy8lVMUMZZ7NIGP01yuJCKF4iM8T5EiVOE=",
    "aic_base_url": "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2",
    "aic_auth_url": "https://sap-build-training-hcd2uswp.authentication.us10.hana.ondemand.com/oauth/token",
    "aic_resource_group": "default",
    "foundation_model": "gpt-4.1"  # Use the foundation model as needed
}

orch_model_params = {
    "orch_url": "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/ddaae0b631e78184",
    "orch_model": "anthropic--claude-4-sonnet",
    "parameters": {
        "temperature": 0.5,
        "max_tokens": 20000,
        "top_p": 0.9
    }
}

In [ ]:
from llm_client import CL_Foundation_Service

obj_llm = CL_Foundation_Service(aic_config)
obj_llm.invoke_llm("Capital of India?", aic_config["foundation_model"],0.7)

INFO:httpx:HTTP Request: POST https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/ded07dd502336fdf/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


'The capital of India is **New Delhi**.'

In [8]:
prompt_gen = cl_msds_Ontology_PromptGenerator("msds_ontology.ttl")
prompt = prompt_gen.generate_prompt(chunks)
print(prompt)

# Knowledge Graph Extraction Instructions for MSDS

## 1. Overview
You are an expert system for extracting RDF triples from Material Safety Data Sheets (MSDS) using the provided ontology. Strictly follow the ontology structure and allowed attributes.

## 2. SDS Sections
Extract information for each of the following 16 SDS sections:
SafetyDataSheet, Identification, HazardsIdentification, CompositionInformationOnIngredients, FirstAidMeasures, FireFightingMeasures, AccidentalReleaseMeasures, HandlingAndStorage, ExposureControlsPersonalProtection, PhysicalAndChemicalProperties, StabilityAndReactivity, ToxicologicalInformation, EcologicalInformation, DisposalConsiderations, TransportationInformation, RegulatoryInformation, OtherInformation, Manufacturer, Ingredient, ExposureLimit, Property

## 3. Allowed Attributes
Only extract properties that match the following allowed attributes (case-insensitive, partial match allowed):
Safety Data Sheet, Identification, Hazards Identification, Composit

### 4. Initialize the Orchetration Model `anthropic--claude-4-sonnet` model 

In [13]:
from langchain.indexes.graph import GraphIndexCreator
from llm_client import CL_Orchestration_Service

obj_orch_client = CL_Orchestration_Service(aic_config, orch_model_params)
chat_llm = obj_orch_client.get_orch_llm_client()
graph_index_creator = GraphIndexCreator(
    llm=chat_llm,
    prompt=prompt,
    ontology=sds_ontology
)


_IncompleteInputError: incomplete input (573570854.py, line 9)

In [9]:
from llm_client import CL_Orchestration_Service
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


obj_orch_client = CL_Orchestration_Service(aic_config, orch_model_params)

# Run orchestration with the prompt and model parameters
try:
    result = obj_orch_client.run_orchestration(prompt, error_context="MSDS_Analysis")
    print("Orchestration Result:")
    print(result)
except Exception as e:
    logger.error("Error during orchestration: %s", e)
    print(f"Error during orchestration: {e}")

KeyboardInterrupt: 